In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/15 07:46:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [3]:
df_green.count()

570466

In [4]:
df_green.createOrReplaceTempView('green')

In [5]:
df_green.columns

['VendorID',
 'lpep_pickup_datetime',
 'lpep_dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'ehail_fee',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'trip_type',
 'congestion_surcharge']

In [6]:
df_green_revenue = spark.sql("""
SELECT
    date_trunc('hour', lpep_pickup_datetime) AS hour,
    PULocationID AS zone,
    SUM(total_amount) AS amount,
    COUNT(1) as number_records
FROM green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY 1, 2
ORDER BY 1, 2
""")

In [7]:
df_green_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-12-31 23:00:00| 168|               9.3|             1|
|2021-01-01 00:00:00|   7|             61.47|             2|
|2021-01-01 00:00:00|  17|            102.34|             3|
|2021-01-01 00:00:00|  35|              50.3|             2|
|2021-01-01 00:00:00|  39|              36.0|             1|
|2021-01-01 00:00:00|  41|               8.3|             1|
|2021-01-01 00:00:00|  42|             28.02|             3|
|2021-01-01 00:00:00|  43|               6.8|             1|
|2021-01-01 00:00:00|  47|             63.19|             2|
|2021-01-01 00:00:00|  55|             57.25|             1|
|2021-01-01 00:00:00|  61|43.760000000000005|             2|
|2021-01-01 00:00:00|  63|             41.99|             1|
|2021-01-01 00:00:00|  74|             86.57|             7|
|2021-01-01 00:00:00|  7

In [8]:
df_green_revenue.write.parquet('data/report/revenue/green', mode='overwrite')

In [10]:
df_green_revenue.count()

251051

In [11]:
!ls -lhR data/report/revenue/green

data/report/revenue/green:
total 1.1M
-rw-r--r-- 1 sweantu sweantu    0 Aug 15 07:48 _SUCCESS
-rw-r--r-- 1 sweantu sweantu 530K Aug 15 07:48 part-00000-ab47b645-9c13-461d-8daa-bfc38006705e-c000.snappy.parquet
-rw-r--r-- 1 sweantu sweantu 543K Aug 15 07:48 part-00001-ab47b645-9c13-461d-8daa-bfc38006705e-c000.snappy.parquet


In [12]:
df_green_revenue.show()

[Stage 21:>                                                         (0 + 2) / 2]

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2021-06-18 20:00:00| 116|22.200000000000003|             2|
|2021-06-29 17:00:00| 181|            106.12|             4|
|2021-06-22 09:00:00|  95|176.21999999999997|             5|
|2021-06-26 17:00:00|   7|            116.63|             8|
|2021-06-13 09:00:00|  25|             20.36|             1|
|2021-06-08 08:00:00|  86| 94.05000000000001|             2|
|2021-06-07 18:00:00|  41|111.88999999999999|             9|
|2021-06-27 13:00:00|  74| 356.4200000000001|            25|
|2021-06-11 22:00:00| 159|               7.3|             1|
|2021-06-25 17:00:00|  65| 97.67999999999999|             4|
|2021-06-10 10:00:00|  75|139.86999999999998|            12|
|2021-06-16 22:00:00| 242|             22.01|             1|
|2021-06-19 21:00:00| 126|              31.3|             1|
|2021-06-17 10:00:00| 20

In [13]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode='overwrite')

In [14]:
!ls -lhR data/report/revenue/green

data/report/revenue/green:
total 2.2M
-rw-r--r-- 1 sweantu sweantu    0 Aug 15 07:52 _SUCCESS
-rw-r--r-- 1 sweantu sweantu 105K Aug 15 07:52 part-00000-bab9106f-febe-447e-ab5a-02cc23883009-c000.snappy.parquet
-rw-r--r-- 1 sweantu sweantu 105K Aug 15 07:52 part-00001-bab9106f-febe-447e-ab5a-02cc23883009-c000.snappy.parquet
-rw-r--r-- 1 sweantu sweantu 105K Aug 15 07:52 part-00002-bab9106f-febe-447e-ab5a-02cc23883009-c000.snappy.parquet
-rw-r--r-- 1 sweantu sweantu 105K Aug 15 07:52 part-00003-bab9106f-febe-447e-ab5a-02cc23883009-c000.snappy.parquet
-rw-r--r-- 1 sweantu sweantu 105K Aug 15 07:52 part-00004-bab9106f-febe-447e-ab5a-02cc23883009-c000.snappy.parquet
-rw-r--r-- 1 sweantu sweantu 105K Aug 15 07:52 part-00005-bab9106f-febe-447e-ab5a-02cc23883009-c000.snappy.parquet
-rw-r--r-- 1 sweantu sweantu 105K Aug 15 07:52 part-00006-bab9106f-febe-447e-ab5a-02cc23883009-c000.snappy.parquet
-rw-r--r-- 1 sweantu sweantu 105K Aug 15 07:52 part-00007-bab9106f-febe-447e-ab5a-02cc23883009-c000.s

In [15]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')

In [16]:
df_yellow.createOrReplaceTempView('yellow')

In [17]:
df_yellow.columns

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge']

In [18]:
df_yellow_revenue = spark.sql("""
SELECT
    date_trunc('hour', tpep_pickup_datetime) AS hour,
    PULocationID AS zone,
    SUM(total_amount) AS amount,
    COUNT(1) as number_records
FROM yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY 1, 2
""")

In [19]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

In [20]:
df_yellow_revenue.count()

583110

In [21]:
!ls -lhR data/report/

data/report/:
total 4.0K
drwxr-xr-x 4 sweantu sweantu 4.0K Aug 15 07:53 revenue

data/report/revenue:
total 208K
-rw-r--r-- 1 sweantu sweantu    0 Aug 15 07:38 _SUCCESS
drwxr-xr-x 2 sweantu sweantu 4.0K Aug 15 07:52 green
-rw-r--r-- 1 sweantu sweantu 198K Aug 15 07:38 part-00000-bc059580-bfbe-45a4-84d3-413c88984aba-c000.snappy.parquet
drwxr-xr-x 2 sweantu sweantu 4.0K Aug 15 07:53 yellow

data/report/revenue/green:
total 2.2M
-rw-r--r-- 1 sweantu sweantu    0 Aug 15 07:52 _SUCCESS
-rw-r--r-- 1 sweantu sweantu 105K Aug 15 07:52 part-00000-bab9106f-febe-447e-ab5a-02cc23883009-c000.snappy.parquet
-rw-r--r-- 1 sweantu sweantu 105K Aug 15 07:52 part-00001-bab9106f-febe-447e-ab5a-02cc23883009-c000.snappy.parquet
-rw-r--r-- 1 sweantu sweantu 105K Aug 15 07:52 part-00002-bab9106f-febe-447e-ab5a-02cc23883009-c000.snappy.parquet
-rw-r--r-- 1 sweantu sweantu 105K Aug 15 07:52 part-00003-bab9106f-febe-447e-ab5a-02cc23883009-c000.snappy.parquet
-rw-r--r-- 1 sweantu sweantu 105K Aug 15 07:52 part-00

In [22]:
df_join = df_green_revenue.join(df_yellow_revenue, on=['hour', 'zone'], how='outer')

In [23]:
df_join.show()

[Stage 47:>                                                         (0 + 1) / 1]

+-------------------+----+------------------+--------------+------------------+--------------+
|               hour|zone|            amount|number_records|            amount|number_records|
+-------------------+----+------------------+--------------+------------------+--------------+
|2020-12-31 18:00:00|  48|              NULL|          NULL|             55.14|             3|
|2020-12-31 18:00:00| 142|              NULL|          NULL|             31.46|             2|
|2020-12-31 21:00:00|  75|              NULL|          NULL|               5.3|             1|
|2020-12-31 21:00:00| 263|              NULL|          NULL|              12.8|             1|
|2020-12-31 23:00:00|  42|              NULL|          NULL|              24.8|             1|
|2020-12-31 23:00:00|  43|              NULL|          NULL|              43.1|             1|
|2020-12-31 23:00:00| 231|              NULL|          NULL|              28.3|             1|
|2021-01-01 00:00:00|   4|              NULL|     

In [24]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')


In [25]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [26]:
df_join

DataFrame[hour: timestamp, zone: int, green_amount: double, green_number_records: bigint, yellow_amount: double, yellow_number_records: bigint]

In [27]:
df_join.show()

[Stage 52:>                                                         (0 + 1) / 1]

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-12-31 18:00:00|  48|              NULL|                NULL|             55.14|                    3|
|2020-12-31 18:00:00| 142|              NULL|                NULL|             31.46|                    2|
|2020-12-31 21:00:00|  75|              NULL|                NULL|               5.3|                    1|
|2020-12-31 21:00:00| 263|              NULL|                NULL|              12.8|                    1|
|2020-12-31 23:00:00|  42|              NULL|                NULL|              24.8|                    1|
|2020-12-31 23:00:00|  43|              NULL|                NULL|              43.1|                    1|
|2020-12-31 23:00:00| 231|  

In [28]:
df_join.repartition(200).write.parquet('data/report/revenue/total', mode='overwrite')

In [29]:
df_join.count()

669604

In [30]:
!ls -lhR 'data/report/revenue/total'

data/report/revenue/total:
total 11M
-rw-r--r-- 1 sweantu sweantu   0 Aug 15 08:03 _SUCCESS
-rw-r--r-- 1 sweantu sweantu 50K Aug 15 08:03 part-00000-efc8c6a0-5286-4696-afb5-c3e4acdfb968-c000.snappy.parquet
-rw-r--r-- 1 sweantu sweantu 50K Aug 15 08:03 part-00001-efc8c6a0-5286-4696-afb5-c3e4acdfb968-c000.snappy.parquet
-rw-r--r-- 1 sweantu sweantu 49K Aug 15 08:03 part-00002-efc8c6a0-5286-4696-afb5-c3e4acdfb968-c000.snappy.parquet
-rw-r--r-- 1 sweantu sweantu 50K Aug 15 08:03 part-00003-efc8c6a0-5286-4696-afb5-c3e4acdfb968-c000.snappy.parquet
-rw-r--r-- 1 sweantu sweantu 50K Aug 15 08:03 part-00004-efc8c6a0-5286-4696-afb5-c3e4acdfb968-c000.snappy.parquet
-rw-r--r-- 1 sweantu sweantu 50K Aug 15 08:03 part-00005-efc8c6a0-5286-4696-afb5-c3e4acdfb968-c000.snappy.parquet
-rw-r--r-- 1 sweantu sweantu 50K Aug 15 08:03 part-00006-efc8c6a0-5286-4696-afb5-c3e4acdfb968-c000.snappy.parquet
-rw-r--r-- 1 sweantu sweantu 50K Aug 15 08:03 part-00007-efc8c6a0-5286-4696-afb5-c3e4acdfb968-c000.snappy.parq

In [31]:
df_green_revenue = spark.read.parquet('data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow')

In [32]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [33]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [34]:
df_join.count()

669604

In [35]:
df_join.repartition(200).write.parquet('data/report/revenue/total', mode='overwrite')

In [36]:
!ls -lhR 'data/report/revenue/total'

data/report/revenue/total:
total 11M
-rw-r--r-- 1 sweantu sweantu   0 Aug 15 08:05 _SUCCESS
-rw-r--r-- 1 sweantu sweantu 50K Aug 15 08:05 part-00000-1095f2b8-5a76-4c3b-b7bc-cf2c3110affb-c000.snappy.parquet
-rw-r--r-- 1 sweantu sweantu 50K Aug 15 08:05 part-00001-1095f2b8-5a76-4c3b-b7bc-cf2c3110affb-c000.snappy.parquet
-rw-r--r-- 1 sweantu sweantu 49K Aug 15 08:05 part-00002-1095f2b8-5a76-4c3b-b7bc-cf2c3110affb-c000.snappy.parquet
-rw-r--r-- 1 sweantu sweantu 50K Aug 15 08:05 part-00003-1095f2b8-5a76-4c3b-b7bc-cf2c3110affb-c000.snappy.parquet
-rw-r--r-- 1 sweantu sweantu 50K Aug 15 08:05 part-00004-1095f2b8-5a76-4c3b-b7bc-cf2c3110affb-c000.snappy.parquet
-rw-r--r-- 1 sweantu sweantu 50K Aug 15 08:05 part-00005-1095f2b8-5a76-4c3b-b7bc-cf2c3110affb-c000.snappy.parquet
-rw-r--r-- 1 sweantu sweantu 50K Aug 15 08:05 part-00006-1095f2b8-5a76-4c3b-b7bc-cf2c3110affb-c000.snappy.parquet
-rw-r--r-- 1 sweantu sweantu 50K Aug 15 08:05 part-00007-1095f2b8-5a76-4c3b-b7bc-cf2c3110affb-c000.snappy.parq

In [37]:
df_join = spark.read.parquet('data/report/revenue/total')

In [38]:
df_join

DataFrame[hour: timestamp, zone: int, green_amount: double, green_number_records: bigint, yellow_amount: double, yellow_number_records: bigint]

In [39]:
!wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv

--2025-08-15 08:07:36--  https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.160.201.50, 18.160.201.126, 18.160.201.131, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.160.201.50|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12331 (12K) [text/csv]
Saving to: ‘taxi_zone_lookup.csv’

taxi_zone_lookup.cs 100%[===================>]  12.04K  --.-KB/s    in 0s      

2025-08-15 08:07:36 (119 MB/s) - ‘taxi_zone_lookup.csv’ saved [12331/12331]



In [40]:
df = spark.read \
    .option("header", "true") \
    .csv('taxi_zone_lookup.csv')

In [41]:
df.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [42]:
df.write.parquet('zones')

In [43]:
df_zones = spark.read.parquet('zones')

In [44]:
df_zones

DataFrame[LocationID: string, Borough: string, Zone: string, service_zone: string]

In [45]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [46]:
df_result.drop('zone', 'LocationID').write.parquet('tmp/revenue-zones', mode='overwrite')

In [47]:
!ls -lh tmp/revenue-zones/

total 6.4M
-rw-r--r-- 1 sweantu sweantu     0 Aug 15 08:15 _SUCCESS
-rw-r--r-- 1 sweantu sweantu  1.1M Aug 15 08:15 part-00000-47228c72-8e7a-467f-8eba-6bdd9f0b5024-c000.snappy.parquet
-rw-r--r-- 1 sweantu sweantu  1.1M Aug 15 08:15 part-00001-47228c72-8e7a-467f-8eba-6bdd9f0b5024-c000.snappy.parquet
-rw-r--r-- 1 sweantu sweantu  1.1M Aug 15 08:15 part-00002-47228c72-8e7a-467f-8eba-6bdd9f0b5024-c000.snappy.parquet
-rw-r--r-- 1 sweantu sweantu  1.1M Aug 15 08:15 part-00003-47228c72-8e7a-467f-8eba-6bdd9f0b5024-c000.snappy.parquet
-rw-r--r-- 1 sweantu sweantu 1022K Aug 15 08:15 part-00004-47228c72-8e7a-467f-8eba-6bdd9f0b5024-c000.snappy.parquet
-rw-r--r-- 1 sweantu sweantu 1021K Aug 15 08:15 part-00005-47228c72-8e7a-467f-8eba-6bdd9f0b5024-c000.snappy.parquet
-rw-r--r-- 1 sweantu sweantu  293K Aug 15 08:15 part-00006-47228c72-8e7a-467f-8eba-6bdd9f0b5024-c000.snappy.parquet


In [48]:
df_result.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+----------+---------+--------------------+------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|LocationID|  Borough|                Zone|service_zone|
+-------------------+----+------------------+--------------------+------------------+---------------------+----------+---------+--------------------+------------+
|2021-06-21 17:00:00| 222|             18.48|                   1|              NULL|                 NULL|       222| Brooklyn|       Starrett City|   Boro Zone|
|2021-06-13 18:00:00| 208|              NULL|                NULL|             22.37|                    1|       208|    Bronx|Schuylerville/Edg...|   Boro Zone|
|2021-04-30 17:00:00| 179|             41.04|                   2|            136.23|                    7|       179|   Queens|         Old Astoria|   Boro Zone|
|2021-05-13 23:00:00| 

In [49]:
df_result.drop('zone', 'LocationID').limit(5).show()

+-------------------+------------+--------------------+------------------+---------------------+---------+------------+
|               hour|green_amount|green_number_records|     yellow_amount|yellow_number_records|  Borough|service_zone|
+-------------------+------------+--------------------+------------------+---------------------+---------+------------+
|2021-06-21 17:00:00|       18.48|                   1|              NULL|                 NULL| Brooklyn|   Boro Zone|
|2021-06-13 18:00:00|        NULL|                NULL|             22.37|                    1|    Bronx|   Boro Zone|
|2021-04-30 17:00:00|       41.04|                   2|            136.23|                    7|   Queens|   Boro Zone|
|2021-05-13 23:00:00|        NULL|                NULL|             66.15|                    3|Manhattan|   Boro Zone|
|2021-07-10 18:00:00|        NULL|                NULL|1584.6099999999994|                   97|Manhattan| Yellow Zone|
+-------------------+------------+------